In [1]:
import pandas as pd
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
from peft import LoraConfig, get_peft_model
from torch import nn
from torch.utils.data.sampler import Sampler
from collections import defaultdict
import random
from tqdm import tqdm


In [2]:
#load the data
trainingset = pd.read_csv('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/selectdata_reduce_faint.csv')

In [3]:
'''stamp_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/stamp_train.npy')
mask_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_train.npy')

mask_gal = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_gal.npy')
mask_annulus = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_annulus.npy')
area_annulus = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/area_annulus.npy')'''

"stamp_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/stamp_train.npy')\nmask_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_train.npy')\n\nmask_gal = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_gal.npy')\nmask_annulus = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_annulus.npy')\narea_annulus = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/area_annulus.npy')"

In [3]:
#The class to arrange the lines of the dataframe
#Can see the below comments

class PairedBatchSampler(Sampler):
    """
    choose P ref_ids, and choose 2 from each ref_id.
    
    P = 8, K = 2 , batch_size = 16
    batch_indices = [idx_A1, idx_G1, ... idx_F1,   idx_A2, idx_G2, ... idx_F2]
                  
    - ref_ids (list)
    - P (int): number of the pair
    """
    def __init__(self, ref_ids, P):
        super(PairedBatchSampler, self).__init__()
        
        if P <= 0:
            raise ValueError("P must be > 0")

        self.P = P
        self.K_is_fixed_at = 2
        self.batch_size = P * self.K_is_fixed_at
        
        print("constructing PairedBatchSampler ...")
        grouped_indices = defaultdict(list)
        for i, ref_id in enumerate(ref_ids):
            grouped_indices[ref_id].append(i)
        
        print(f"creating 'chunks' (size K={self.K_is_fixed_at})...")
        self.all_chunks = []
        for ref_id, indices in grouped_indices.items():
            if len(indices) >= self.K_is_fixed_at: #Acutally I have already selected the ref_ids
                
                random.shuffle(indices) #make it random
                
                # divide the ref_id. e.g. floor(13 / 4) = 3. we build 3 chunks。
                num_chunks_for_this_id = len(indices) // self.K_is_fixed_at
                
                for i in range(num_chunks_for_this_id):
                    chunk = indices[i * self.K_is_fixed_at : (i + 1) * self.K_is_fixed_at]
                    self.all_chunks.append(chunk)
        
        print(f"Already created {len(self.all_chunks)} 'K-chunks'.")
        
        if len(self.all_chunks) < P:
            raise ValueError(f"ref_ids are less than P={P} 'K-chunks'. Use a smaller K or P")
            
        self.num_batches = len(self.all_chunks) // P

    def __iter__(self):
        
        random.shuffle(self.all_chunks)
        
        for i in range(self.num_batches):
            batch_part1_indices = []
            batch_part2_indices = []
            
            p_chunks = self.all_chunks[i * self.P : (i + 1) * self.P]
            
            for chunk in p_chunks:
                
                batch_part1_indices.append(chunk[0])
                batch_part2_indices.append(chunk[1])
            
            final_batch_indices = batch_part1_indices + batch_part2_indices
            
            # P + P = 16
            yield final_batch_indices

    def __len__(self):
        return self.num_batches

In [4]:
#Select and arrange the dataframe
P = 8
sampler = PairedBatchSampler(trainingset['ref_id'], P=P)
print("\n getting index from the sampler...")
list_of_all_batches = list(sampler)

shuffled_indices = [index for batch in list_of_all_batches for index in batch]
print(f"Have created {len(shuffled_indices)} lines")

shuffled_df_train = trainingset.iloc[shuffled_indices]
shuffled_df_train = shuffled_df_train.reset_index(drop=True)

constructing PairedBatchSampler ...
creating 'chunks' (size K=2)...
Already created 14629 'K-chunks'.

 getting index from the sampler...
Have created 29248 lines


In [5]:
#generate image
cutout_size = 48

def background_annulus(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    """Measure background in an annulus."""
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')

    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    S = pd.Series()
    S['annulus_mean'] = np.ma.mean(clip_annulus_array)
    S['annulus_median'] = np.ma.median(clip_annulus_array)
    S['annulus_std'] = np.ma.std(clip_annulus_array)
    S['annulus_samples'] = np.ma.count(clip_annulus_array)

    return S

def flux_elliptical(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    """Measure the flux withing an elliptical aperture."""
    
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)

    xmask = mask != 0
    raw_flux = aperture_photometry(image, source_aperture, mask=xmask)
   
    S = pd.Series()
    S['raw_flux'] = float(raw_flux['aperture_sum'][0])
    S['area'] = source_aperture.area
    
    return S

def cal_calerror(sig_src,sig_zp,zp,f_src):
    sig_cal = np.sqrt(sig_src**2 * sig_zp**2 + sig_src**2 * zp**2 + sig_zp**2 * f_src**2)
    return sig_cal

def cal_fcal(f_src,zp):
    f_cal = f_src * zp
    return f_cal

def creat_stamps(image, sources):
    y = math.floor(sources['aperture_x'].values[0])
    x = math.floor(sources['aperture_y'].values[0])
    #y = math.floor(sources['aperture_x'])
    #x = math.floor(sources['aperture_y'])
    x_start = max((x - cutout_size), 0)
    x_end = min((x + cutout_size), image.shape[0])
    y_start = max((y - cutout_size), 0)
    y_end = min((y + cutout_size), image.shape[1])

    stamps = image[x_start:x_end, y_start:y_end]
    return stamps

def photometry_oneimage(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    
    S1 = background_annulus(image, mask, aperture_x, aperture_y)
    S2 = flux_elliptical(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b)

    flux_obs = S2['raw_flux'] - S2['area'] * S1['annulus_mean']
    return flux_obs, S1['annulus_std']

def generate_mask(size_data, image):

    num_sample = int(size_data[0] * size_data[1] * (1 - ratio))
    mask = np.ones(size_data)
    output = image

    for ich in range(size_data[2]):
        idy_msk = np.random.randint(0, size_data[0], num_sample)
        idx_msk = np.random.randint(0, size_data[1], num_sample)

        idy_neigh = np.random.randint(-size_window[0] // 2 + size_window[0] % 2, size_window[0] // 2 + size_window[0] % 2, num_sample)
        idx_neigh = np.random.randint(-size_window[1] // 2 + size_window[1] % 2, size_window[1] // 2 + size_window[1] % 2, num_sample)

        idy_msk_neigh = idy_msk + idy_neigh
        idx_msk_neigh = idx_msk + idx_neigh

        idy_msk_neigh = idy_msk_neigh + (idy_msk_neigh < 0) * size_data[0] - (idy_msk_neigh >= size_data[0]) * size_data[0]
        idx_msk_neigh = idx_msk_neigh + (idx_msk_neigh < 0) * size_data[1] - (idx_msk_neigh >= size_data[1]) * size_data[1]

        id_msk = (idy_msk, idx_msk, ich)
        id_msk_neigh = (idy_msk_neigh, idx_msk_neigh, ich)

        output[id_msk] = image[id_msk_neigh]
        mask[id_msk] = 0.0

    return output, mask

In [6]:
#write the function to generate the mask_annulus, mask_gal
#test the code in the following
def annulus_mask_generator(mask, aperture_x, aperture_y, r_in=30, r_out=45):

    image_shape = (cutout_size*2,cutout_size*2)
    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    
    mask_object = annulus_apertures.to_mask(method='center')
    mask_annulus = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    
    mask_annulus = mask_annulus * (1 - xmask)
    
    return mask_annulus, annulus_apertures.area

def gal_mask_generator(mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    image_shape = (cutout_size*2,cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    mask_gal = mask_image_photutils_fractional * (1 - xmask)
    
    return mask_gal#, source_aperture.area

In [12]:
#generate the stamp and mask
cutout_size = 48
shuffled_stamp_train = np.zeros((len(shuffled_df_train), cutout_size*2, cutout_size*2))
shuffled_mask_train = np.zeros((len(shuffled_df_train), cutout_size*2, cutout_size*2))

for i in tqdm(range(len(shuffled_df_train))):
    row = shuffled_df_train.iloc[[i]]
    path = shuffled_df_train['path'][i]
    image = fits.getdata(path)
    mask = fits.getdata(path.replace('.fits', '.mask.fits'))
    
    shuffled_stamp_train[i] = creat_stamps(image, row)
    shuffled_mask_train[i] = creat_stamps(mask, row)


shuffled_mask_gal = np.zeros((len(shuffled_df_train), cutout_size*2, cutout_size*2))
shuffled_mask_annulus = np.zeros((len(shuffled_df_train), cutout_size*2, cutout_size*2))
shuffled_area_annulus = np.zeros((len(shuffled_df_train)))

100%|██████████| 29248/29248 [37:58<00:00, 12.84it/s]  


In [13]:
for i in tqdm(range(len(shuffled_df_train))):
    row = shuffled_df_train.iloc[[i]]
    aperture_x = cutout_size + row['aperture_x'].item() - math.floor(row['aperture_x'].item())
    aperture_y = cutout_size + row['aperture_y'].item() - math.floor(row['aperture_y'].item())

    shuffled_mask_gal[i] = gal_mask_generator(shuffled_mask_train[i], aperture_x, aperture_y,
                                   row['aperture_theta'].item(),row['aperture_a'].item(),row['aperture_b'].item())
    shuffled_mask_annulus[i], shuffled_area_annulus[i] = annulus_mask_generator(shuffled_mask_train[i], aperture_x, aperture_y)

100%|██████████| 29248/29248 [01:01<00:00, 474.69it/s]


In [14]:
#This part need to process the mask_abnormal,to (1 - mask_abnormal)

In [15]:
shuffled_mask_train = 1 - shuffled_mask_train

In [16]:
#To convert them into tensor
tensor_stamp_train = torch.from_numpy(shuffled_stamp_train).float()
tensor_mask_train = torch.from_numpy(shuffled_mask_train).float()
tensor_mask_gal = torch.from_numpy(shuffled_mask_gal).float()
tensor_mask_annulus = torch.from_numpy(shuffled_mask_annulus).float()
tensor_area_annulus = torch.from_numpy(shuffled_area_annulus).float()

tensor_stamp_train = tensor_stamp_train.unsqueeze(1)
tensor_mask_train = tensor_mask_train.unsqueeze(1)
tensor_mask_gal = tensor_mask_gal.unsqueeze(1)
tensor_mask_annulus = tensor_mask_annulus.unsqueeze(1)
tensor_area_annulus = tensor_area_annulus.unsqueeze(1)

#We just need 'ref_id', 'zp'
features_df_train = shuffled_df_train[['area', 'ref_id', 'zp']].values
features_df_train_tensor = torch.FloatTensor(features_df_train)

In [17]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(tensor_stamp_train, tensor_mask_train, tensor_mask_gal, tensor_mask_annulus, tensor_area_annulus, features_df_train_tensor)
traindataloader = DataLoader(dataset, batch_size=16, shuffle=False)
#I set shuffle=False here, because I have already do the shuffle before

In [18]:
#I have already simplified the loss functions

def loss_unbias(flux_gal_inputs, flux_gal_outputs):

    loss = (flux_gal_inputs - flux_gal_outputs).abs().sum()
    
    return loss
    
def PairedDifferenceLoss(flux_gal_calibrated_outputs):
    
    batch_size = flux_gal_calibrated_outputs.shape[0]
    half_B = batch_size // 2 #divide into 2
        
    outputs_1 = flux_gal_calibrated_outputs[0:half_B]
    outputs_2 = flux_gal_calibrated_outputs[half_B:]
    loss = (outputs_1 - outputs_2).abs().sum()
    
    return loss

In [26]:
#load the pn2v model
import os
os.chdir('/data/aai/scratch/jchan/denoise/PAUS/dinggetest/simulation/pn2v/src/pn2v')
from core import prediction
from core import utils
from unet import UNet

device=utils.getDevice()

CUDA available? True


In [32]:
path='/data/aai/scratch/jchan/denoise/PAUS/dinggetest/simulation/model saved/'
model=torch.load(path+"/best_conv_N2V_PAUdm.net")

/tmp/ipykernel_63/4130286196.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load(path+"/best_conv_N2V_PAUdm.net")


In [33]:
for param in model.parameters():
    param.requires_grad = False

In [34]:
#print(model)
target_list = [
    'conv_final',
    'down_convs.0.conv1',
    'down_convs.0.conv2',
    'down_convs.1.conv1',
    'down_convs.1.conv2',
    'down_convs.2.conv1',
    'down_convs.2.conv2',
    'up_convs.0.conv1',
    'up_convs.0.conv2',
    'up_convs.1.conv1',
    'up_convs.1.conv2'
]

In [35]:
#I add some Chinese comments here to explain some parameters...

config = LoraConfig(
    r=8,  #它控制了 LoRA 模块的“大小”或“复杂度”。r 越大，LoRA 模块的可训练参数就越多，理论上能学习更复杂的调整，但也会占用更多显存。r=8 或 16 是一个非常常见且高效的选择。
    lora_alpha=16, #LoRA 的输出会乘以一个缩放比例 alpha/r。这就像一个特殊的“学习率”或“平衡旋钮”。一个常见的经验法则是将 lora_alpha 设置为 r 的两倍（比如 r=8, alpha=16），这有助于稳定训练。
    target_modules=target_list,
    lora_dropout=0.1,#在 LoRA 模块中添加一个 Dropout 层，用于防止过拟合，这是一个标准的正则化技术。To prevent overfitting
)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 84,560 || all params: 1,761,938 || trainable%: 4.7993


In [36]:
num_epochs = 200
lora_model.train()
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    
    total_loss1 = 0.0
    total_loss2 = 0.0
    num_batches = 0
    
    progress_bar = tqdm(traindataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
    for tensor_stamp_train, tensor_mask_train, tensor_mask_gal, tensor_mask_annulus, tensor_area_annulus, features_df_train_tensor in progress_bar:

        inputs = tensor_stamp_train.to(device)
        mask_abnormal = tensor_mask_train.to(device) 
        mask_gal = tensor_mask_gal.to(device)
        mask_annulus = tensor_mask_annulus.to(device)
        area_annulus = tensor_area_annulus.to(device)
        features = features_df_train_tensor.to(device)#'area', 'ref_id', 'zp'
        optimizer.zero_grad()
        
        outputs = lora_model(inputs) 
        #batch_size = inputs.shape[0]
        batch_size = inputs.shape[0]
        #get the flux
        flux_raw_inputs = torch.sum(inputs * mask_gal * mask_abnormal, dim=[2, 3])
        flux_raw_outputs = torch.sum(outputs * mask_gal * mask_abnormal, dim=[2, 3])
        flux_annulus_inputs = torch.sum(inputs * mask_annulus, dim=[2, 3])
        flux_annulus_outputs = torch.sum(outputs * mask_annulus, dim=[2, 3])
        flux_gal_inputs = flux_raw_inputs - flux_annulus_inputs * features[:,0].unsqueeze(1) / area_annulus
        flux_gal_outputs = flux_raw_outputs - flux_annulus_outputs * features[:,0].unsqueeze(1) / area_annulus
        #get the calibrated flux.
        #Note that here we just calculate the calibrated flux of the denoised image, we don't need to include the undenoised ones
        flux_gal_calibrated_outputs = flux_gal_outputs * features[:,2].unsqueeze(1)
        
        loss1 = loss_unbias(flux_gal_inputs, flux_gal_outputs) / batch_size
        loss2 = PairedDifferenceLoss(flux_gal_calibrated_outputs) / batch_size#(batch_size/2)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        
        current_loss1 = loss1.item()
        current_loss2 = loss2.item()
        total_loss1 += current_loss1
        total_loss2 += current_loss2
        num_batches += 1
        progress_bar.set_postfix({
            'Loss of this batch': f'{current_loss1 + current_loss2:.6f}',
            'Avg Loss': f'{(current_loss1+current_loss2)/num_batches:.6f}'
        })
    
    avg_loss1 = total_loss1 / num_batches
    avg_loss2 = total_loss2 / num_batches
    
    print(f"\nEpoch {epoch+1} Completed - Average unbias Loss: {avg_loss1:.6f} Average df Loss {avg_loss2:.6f}")

Epoch 1/200: 100%|██████████| 1828/1828 [01:02<00:00, 29.11it/s, Loss of this batch=3.681190, Avg Loss=0.002014] 



Epoch 1 Completed - Average unbias Loss: 2.363595 Average df Loss 1.034137


Epoch 2/200: 100%|██████████| 1828/1828 [01:02<00:00, 29.03it/s, Loss of this batch=3.599239, Avg Loss=0.001969] 



Epoch 2 Completed - Average unbias Loss: 2.176764 Average df Loss 0.983623


Epoch 3/200: 100%|██████████| 1828/1828 [01:02<00:00, 29.20it/s, Loss of this batch=3.744357, Avg Loss=0.002048] 


...


Epoch 9 Completed - Average unbias Loss: 2.134182 Average df Loss 0.770952


Epoch 10/200: 100%|██████████| 1828/1828 [01:02<00:00, 29.23it/s, Loss of this batch=3.485407, Avg Loss=0.001907] 



Epoch 10 Completed - Average unbias Loss: 2.133935 Average df Loss 0.765080


Epoch 11/200:  41%|████▏     | 758/1828 [00:25<00:36, 29.18it/s, Loss of this batch=2.047821, Avg Loss=0.002702]


KeyboardInterrupt: 

In [25]:
save_dir = "/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/saved_models"
final_model_path = os.path.join(save_dir, "lora_model_final_faint3.pth")
torch.save(lora_model, final_model_path)

In [66]:
num_epochs = 200
lora_model.train()
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-5)

for epoch in range(num_epochs):
    
    total_loss1 = 0.0
    total_loss2 = 0.0
    num_batches = 0
    
    progress_bar = tqdm(traindataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
    for tensor_stamp_train, tensor_mask_train, tensor_mask_gal, tensor_mask_annulus, tensor_area_annulus, features_df_train_tensor in progress_bar:

        inputs = tensor_stamp_train.to(device)
        mask_abnormal = tensor_mask_train.to(device) 
        mask_gal = tensor_mask_gal.to(device)
        mask_annulus = tensor_mask_annulus.to(device)
        area_annulus = tensor_area_annulus.to(device)
        features = features_df_train_tensor.to(device)#'area', 'ref_id', 'zp'
        optimizer.zero_grad()
        
        outputs = lora_model(inputs) 
        #batch_size = inputs.shape[0]
        batch_size = inputs.shape[0]
        #get the flux
        flux_raw_inputs = torch.sum(inputs * mask_gal * mask_abnormal, dim=[2, 3])
        flux_raw_outputs = torch.sum(outputs * mask_gal * mask_abnormal, dim=[2, 3])
        flux_annulus_inputs = torch.sum(inputs * mask_annulus, dim=[2, 3])
        flux_annulus_outputs = torch.sum(outputs * mask_annulus, dim=[2, 3])
        flux_gal_inputs = flux_raw_inputs - flux_annulus_inputs * features[:,0].unsqueeze(1) / area_annulus
        flux_gal_outputs = flux_raw_outputs - flux_annulus_outputs * features[:,0].unsqueeze(1) / area_annulus
        #get the calibrated flux.
        #Note that here we just calculate the calibrated flux of the denoised image, we don't need to include the undenoised ones
        flux_gal_calibrated_outputs = flux_gal_outputs * features[:,2].unsqueeze(1)
        
        loss1 = loss_unbias(flux_gal_inputs, flux_gal_outputs) / batch_size
        loss2 = PairedDifferenceLoss(flux_gal_calibrated_outputs) / batch_size#(batch_size/2)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        
        current_loss1 = loss1.item()
        current_loss2 = loss2.item()
        total_loss1 += current_loss1
        total_loss2 += current_loss2
        num_batches += 1
        progress_bar.set_postfix({
            'Loss of this batch': f'{current_loss1 + current_loss2:.6f}',
            'Avg Loss': f'{(current_loss1+current_loss2)/num_batches:.6f}'
        })
    
    avg_loss1 = total_loss1 / num_batches
    avg_loss2 = total_loss2 / num_batches
    
    print(f"\nEpoch {epoch+1} Completed - Average unbias Loss: {avg_loss1:.6f} Average df Loss {avg_loss2:.6f}")

Epoch 1/200: 100%|██████████| 1828/1828 [00:57<00:00, 31.66it/s, Loss of this batch=2.400738, Avg Loss=0.001313] 



Epoch 1 Completed - Average unbias Loss: 2.505855 Average df Loss 0.993405


Epoch 2/200: 100%|██████████| 1828/1828 [00:57<00:00, 31.64it/s, Loss of this batch=2.396783, Avg Loss=0.001311] 



Epoch 2 Completed - Average unbias Loss: 2.202726 Average df Loss 1.097579


Epoch 3/200: 100%|██████████| 1828/1828 [00:57<00:00, 31.63it/s, Loss of this batch=2.488276, Avg Loss=0.001361] 


...

Epoch 190/200:  98%|█████████▊| 1783/1828 [00:57<00:01, 31.22it/s, Loss of this batch=2.093387, Avg Loss=0.001172] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 195/200: 100%|██████████| 1828/1828 [00:58<00:00, 31.35it/s, Loss of this batch=1.981653, Avg Loss=0.001084] 



Epoch 195 Completed - Average unbias Loss: 2.088650 Average df Loss 0.686157


Epoch 196/200: 100%|██████████| 1828/1828 [00:57<00:00, 31.53it/s, Loss of this batch=2.015147, Avg Loss=0.001102] 



Epoch 196 Completed - Average unbias Loss: 2.087490 Average df Loss 0.685858


Epoch 197/200:  95%|█████████▍| 1732/1828 [00:55<00:03, 31.40it/s, Loss of this batch=2.619119, Avg Loss=0.001511] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
def background_annulus(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    """Measure background in an annulus."""
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')

    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    S = pd.Series()
    S['annulus_mean'] = np.ma.mean(clip_annulus_array)
    S['annulus_median'] = np.ma.median(clip_annulus_array)
    S['annulus_std'] = np.ma.std(clip_annulus_array)
    S['annulus_samples'] = np.ma.count(clip_annulus_array)

    return S

def flux_elliptical(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    """Measure the flux withing an elliptical aperture."""
    
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)

    xmask = mask != 0
    raw_flux = aperture_photometry(image, source_aperture, mask=xmask)
   
    S = pd.Series()
    S['raw_flux'] = float(raw_flux['aperture_sum'][0])
    S['area'] = source_aperture.area
    
    return S

def cal_calerror(sig_src,sig_zp,zp,f_src):
    sig_cal = np.sqrt(sig_src**2 * sig_zp**2 + sig_src**2 * zp**2 + sig_zp**2 * f_src**2)
    return sig_cal

def cal_fcal(f_src,zp):
    f_cal = f_src * zp
    return f_cal

def creat_stamps(image, sources):
    y = math.floor(sources['aperture_x'].values[0])
    x = math.floor(sources['aperture_y'].values[0])
    #y = math.floor(sources['aperture_x'])
    #x = math.floor(sources['aperture_y'])
    x_start = max((x - cutout_size), 0)
    x_end = min((x + cutout_size), image.shape[0])
    y_start = max((y - cutout_size), 0)
    y_end = min((y + cutout_size), image.shape[1])

    stamps = image[x_start:x_end, y_start:y_end]
    return stamps

def photometry_oneimage(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
    
    S1 = background_annulus(image, mask, aperture_x, aperture_y)
    S2 = flux_elliptical(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b)

    flux_obs = S2['raw_flux'] - S2['area'] * S1['annulus_median']
    return flux_obs, S1['annulus_std']

def generate_mask(size_data, image):

    num_sample = int(size_data[0] * size_data[1] * (1 - ratio))
    mask = np.ones(size_data)
    output = image

    for ich in range(size_data[2]):
        idy_msk = np.random.randint(0, size_data[0], num_sample)
        idx_msk = np.random.randint(0, size_data[1], num_sample)

        idy_neigh = np.random.randint(-size_window[0] // 2 + size_window[0] % 2, size_window[0] // 2 + size_window[0] % 2, num_sample)
        idx_neigh = np.random.randint(-size_window[1] // 2 + size_window[1] % 2, size_window[1] // 2 + size_window[1] % 2, num_sample)

        idy_msk_neigh = idy_msk + idy_neigh
        idx_msk_neigh = idx_msk + idx_neigh

        idy_msk_neigh = idy_msk_neigh + (idy_msk_neigh < 0) * size_data[0] - (idy_msk_neigh >= size_data[0]) * size_data[0]
        idx_msk_neigh = idx_msk_neigh + (idx_msk_neigh < 0) * size_data[1] - (idx_msk_neigh >= size_data[1]) * size_data[1]

        id_msk = (idy_msk, idx_msk, ich)
        id_msk_neigh = (idy_msk_neigh, idx_msk_neigh, ich)

        output[id_msk] = image[id_msk_neigh]
        mask[id_msk] = 0.0

    return output, mask

In [10]:
#write the function to calculate the flux by myself

def background_annulus_jiefeng(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')
    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    #background_annulus = np.ma.mean(clip_annulus_array)
    #we use median here, in the dataset they use mdian
    background_annulus = np.ma.median(clip_annulus_array)
    return background_annulus

def flux_elliptical_jiefeng(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    image_shape = (cutout_size*2,cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    image_good = image * (1 - xmask)
    
    raw_flux = np.sum(image_good * mask_image_photutils_fractional)#calculate by myself

    background = background_annulus_jiefeng(image, mask, aperture_x, aperture_y)
    
    gal_flux = raw_flux - source_aperture.area * background
    return gal_flux

In [11]:
from astropy.io import fits
from photutils import CircularAnnulus, EllipticalAperture
from astropy.stats import sigma_clip
from photutils.aperture import aperture_photometry


/tmp/ipykernel_63/1489632568.py:2: DeprecationWarning: `photutils.CircularAnnulus` is a deprecated alias for `photutils.aperture.CircularAnnulus` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAnnulus` to silence this warning.
  from photutils import CircularAnnulus, EllipticalAperture
/tmp/ipykernel_63/1489632568.py:2: DeprecationWarning: `photutils.EllipticalAperture` is a deprecated alias for `photutils.aperture.EllipticalAperture` and will be removed in the future. Instead, please use `from photutils.aperture import EllipticalAperture` to silence this warning.
  from photutils import CircularAnnulus, EllipticalAperture


In [32]:
area = shuffled_df_train[['area']].values
#area = trainingset[['area']].values

In [42]:
num = 30
raw = np.sum(shuffled_stamp_train[num] * shuffled_mask_train[num] * shuffled_mask_gal[num])
bgk_mean = np.sum(shuffled_stamp_train[num] * shuffled_mask_train[num] * shuffled_mask_annulus[num]) / shuffled_area_annulus[num]
gal = raw - bgk_mean * area[num]

print(gal)

[12.93264236]


In [43]:
row = shuffled_df_train.iloc[[num]]
path = shuffled_df_train['path'][num]
image = fits.getdata(path)
mask = fits.getdata(path.replace('.fits', '.mask.fits'))

cutout_size = 48
aperture_x = cutout_size + row['aperture_x'].item() - math.floor(row['aperture_x'].item())
aperture_y = cutout_size + row['aperture_y'].item() - math.floor(row['aperture_y'].item())

stamp_image = creat_stamps(image, row)
mask_cutout = creat_stamps(mask, row)

gal_flux = flux_elliptical_jiefeng(stamp_image, mask_cutout, aperture_x, aperture_y,
                                   row['aperture_theta'].item(),row['aperture_a'].item(),row['aperture_b'].item())
print(gal_flux)
#print(row['area'].item())

12.893120548310137
